In [1]:
import tensorflow as tf
import numpy      as np
import pandas     as pd
import seaborn    as sns
from sklearn.model_selection   import train_test_split
import matplotlib.pyplot as plt
import missingno as msno

In [2]:
airPressure_area_frm = pd.read_csv('../content/airPressure_area_0711.csv', encoding='cp949')
humidity_area_frm = pd.read_csv('../content/humidity_area_0711.csv', encoding='cp949')
rain_area_frm = pd.read_csv('../content/rain_area_0711.csv', encoding='cp949')
sun_area_frm = pd.read_csv('../content/sun_area_0711.csv', encoding='cp949')
temp_area_frm = pd.read_csv('../content/temp_area_0711.csv', encoding='cp949')

In [3]:
airPressure_area_frm.head()

,area,tma,yyyy,mm,dd,stn_id,avg_ps,max_ps,min_ps
0,전남,2011-12-01,2011,12,1,301,NaN,NaN,NaN
1,전남,2011-12-01,2011,12,1,305,1021.4,1022.9,1020.0
2,제주,2011-12-01,2011,12,1,328,NaN,NaN,NaN
3,서울,2011-12-01,2011,12,1,402,NaN,NaN,NaN
4,서울,2011-12-01,2011,12,1,406,NaN,NaN,NaN


In [4]:
humidity_area_frm.head()

,area,tma,yyyy,mm,dd,stn_id,avg_rhm,min_rhm
0,전남,2011-12-01,2011,12,1,305,NaN,NaN
1,경북,2011-12-01,2011,12,1,312,NaN,62.8
2,경남,2011-12-01,2011,12,1,313,76.0,67.7
3,전남,2011-12-01,2011,12,1,315,100.0,99.9
4,강원,2011-12-01,2011,12,1,318,NaN,NaN


In [5]:
rain_area_frm.head()

,area,tma,yyyy,mm,dd,stn_id,sum_rn
0,전남,2011-12-01,2011,12,1,303,11.5
1,강원,2011-12-01,2011,12,1,321,0.0
2,충북,2011-12-01,2011,12,1,325,0.0
3,충북,2011-12-01,2011,12,1,327,0.5
4,서울,2011-12-01,2011,12,1,405,0.0


In [6]:
sun_area_frm.head()

,area,tma,yyyy,mm,dd,stn_id,sum_gsr
0,충남,2011-12-01,2011,12,1,12,0.0
1,경기,2011-12-01,2011,12,1,116,0.0
2,전북,2011-12-01,2011,12,1,300,0.0
3,광주,2011-12-01,2011,12,1,316,0.0
4,강원,2011-12-01,2011,12,1,320,0.0


In [7]:
temp_area_frm.head()

,area,tma,yyyy,mm,dd,stn_id,avg_ta,max_ta,min_ta
0,전북,2011-12-01,2011,12,1,300,7.5,9.2,5.6
1,경기,2011-12-01,2011,12,1,326,4.6,6.9,3.0
2,서울,2011-12-01,2011,12,1,403,7.0,10.2,5.2
3,서울,2011-12-01,2011,12,1,416,5.9,9.3,3.8
4,서울,2011-12-01,2011,12,1,424,6.1,8.7,4.5


In [8]:
# # Unnamed: 0 index 삭제
# del airPressure_area_frm["Unnamed: 0"]
# del humidity_area_frm["Unnamed: 0"]
# del rain_area_frm["Unnamed: 0"]
# del sun_area_frm["Unnamed: 0"]
# del temp_area_frm["Unnamed: 0"]

#### 지점 드랍 후 지역 일시 *groupby*

In [9]:
# 기압 지점 드랍 후 지역, 일시 groupby
airPressure_area_grp = airPressure_area_frm.drop('stn_id',axis=1).groupby(['area',"tma"], as_index = False).mean()

# 습도 지점 드랍 후 지역, 일시 groupby
humidity_area_grp = humidity_area_frm.drop('stn_id',axis=1).groupby(['area',"tma"], as_index = False).mean()

# 강수량 지점 드랍 후 지역, 일시 groupby
rain_area_grp = rain_area_frm.drop('stn_id',axis=1).groupby(['area',"tma"], as_index = False).mean()

# 일사량 지점 드랍 후 지역, 일시 groupby
sun_area_grp = sun_area_frm.drop('stn_id',axis=1).groupby(['area',"tma"], as_index = False).mean()

# 기온 지점 드랍 후 지역, 일시 groupby
temp_area_grp = temp_area_frm.drop('stn_id',axis=1).groupby(['area',"tma"], as_index = False).mean()



In [10]:
# airPressure, humidity, rain, sun, temp          
# row 31586으로 딱 떨어짐.                        
# 얘네 가지고 결측치 채움.                         

In [11]:
# # 로직 : airPressure_area_frm의 결측치 부분의 일자와 지역이 airPressure_area_grp의 부분과 같으면 airPressure_area_grp의 값으로 대체. 
# for i in range(1246362):                                                                                                             
#   if (airPressure_area_frm["avg_ps"][i].isnull == True) and (airPressure_area_frm["tma"] == airPressure_area_grp["tma"] and (airPressure_area_frm["area"] == airPressure_area_grp["area"])):
#     airPressure_area_frm["avg_ps"] = airPressure_area_grp["avg_ps"]
                                                                     
# 집 가서 다시 보겠습니다.

In [12]:
# "csv파일"_area_grp.head()

#### 결측치 지역평균 및 max-min처리

In [13]:
# rain 결측치 지역평균으로 대체 -> 기상데이터를 지역 기준으로 보기때문에 지역 평균으로 대체    
rain_avg_by_date = rain_area_frm.pivot_table(index = 'tma', values = 'sum_rn', aggfunc='mean') 

rain_fill = rain_area_frm.apply(lambda frm :  rain_avg_by_date.loc[frm['tma'], 'sum_rn'] if (frm['sum_rn'] != frm['sum_rn']) 
                  else frm['sum_rn'], axis=1)

rain_area_frm['sum_rn'] = rain_fill

In [14]:
# temp_max 결측치 지역평균으로 대체 -> 기상데이터를 지역 기준으로 보기때문에 지역 평균으로 대체
temp_max_by_date = temp_area_frm.pivot_table(index = 'tma', values = 'max_ta', aggfunc='mean')

temp_max_fill = temp_area_frm.apply(lambda frm :  temp_max_by_date.loc[frm['tma'], 'max_ta'] if (frm['max_ta'] != frm['max_ta']) 
                  else frm['max_ta'], axis=1)

temp_area_frm['max_ta'] = temp_max_fill

In [15]:
# temp_min 결측치 지역평균으로 대체 -> 기상데이터를 지역 기준으로 보기때문에 지역 평균으로 대체
temp_min_by_date = temp_area_frm.pivot_table(index = 'tma', values = 'min_ta', aggfunc='mean')

temp_min_fill = temp_area_frm.apply(lambda frm :  temp_min_by_date.loc[frm['tma'], 'min_ta'] if (frm['min_ta'] != frm['min_ta']) 
                  else frm['min_ta'], axis=1)
temp_area_frm['min_ta'] = temp_min_fill

In [16]:
# temp_avg 결측치 지역평균으로 대체 -> 한시간마다 측정한 기온값들 다 더해서 24로 나눈것이 temp avg값이라 대체
temp_avg_by_date = temp_area_frm.pivot_table(index = 'tma', values = 'avg_ta', aggfunc='mean')

temp_avg_fill = temp_area_frm.apply(lambda frm :  temp_avg_by_date.loc[frm['tma'], 'avg_ta'] if (frm['avg_ta'] != frm['avg_ta']) 
                  else frm['avg_ta'], axis=1)
temp_area_frm['avg_ta'] = temp_avg_fill

In [17]:
#결측값 확인
print("airPressure_area_frm.isnull")
print(airPressure_area_frm.isnull().sum())
print("*"*50)
print()
print("humidity_area_frm.isnull")
print(humidity_area_frm.isnull().sum())
print("*"*50)
print()
print("rain_area_frm.isnull")
print(rain_area_frm.isnull().sum())
print("*"*50)
print()
print("sun_area_frm.isnull")
print(sun_area_frm.isnull().sum())
print("*"*50)
print()
print("temp_area_frm.isnull")
print(temp_area_frm.isnull().sum())

airPressure_area_frm.isnull
area           0
tma            0
yyyy           0
mm             0
dd             0
stn_id         0
avg_ps    658173
max_ps    636785
min_ps    636879
dtype: int64
**************************************************

humidity_area_frm.isnull
area            0
tma             0
yyyy            0
mm              0
dd              0
stn_id          0
avg_rhm    433698
min_rhm    420532
dtype: int64
**************************************************

rain_area_frm.isnull
area      0
tma       0
yyyy      0
mm        0
dd        0
stn_id    0
sum_rn    0
dtype: int64
**************************************************

sun_area_frm.isnull
area       0
tma        0
yyyy       0
mm         0
dd         0
stn_id     0
sum_gsr    0
dtype: int64
**************************************************

temp_area_frm.isnull
area      0
tma       0
yyyy      0
mm        0
dd        0
stn_id    0
avg_ta    0
max_ta    0
min_ta    0
dtype: int64


In [49]:
# merge 사전작업 "area", "mm","dd","yyyy" drop
sun_area_grp.drop(labels = None, columns=["tma","area","mm","dd","yyyy"], axis =1 ,inplace = True)
rain_area_grp.drop(labels = None, columns=["tma","area","mm","dd","yyyy"], axis =1 ,inplace = True)

In [54]:
merge = pd.concat([temp_area_grp, sun_area_grp, rain_area_grp], axis = 1)
merge

,area,tma,yyyy,mm,dd,avg_ta,max_ta,min_ta,sum_gsr,sum_rn
0,강원,2011-12-01,2011.0,12.0,1.0,1.281944,3.573239,-0.271831,0.108493,3.038889
1,강원,2011-12-02,2011.0,12.0,2.0,2.249315,4.736111,-0.347222,0.383151,4.562500
2,강원,2011-12-03,2011.0,12.0,3.0,2.049315,4.409722,0.029167,0.349041,13.219444
3,강원,2011-12-04,2011.0,12.0,4.0,0.513699,4.963889,-3.408333,0.918082,0.007042
4,강원,2011-12-05,2011.0,12.0,5.0,-1.587671,4.665278,-6.556944,0.947808,0.000000
...,...,...,...,...,...,...,...,...,...,...
31581,충북,2016-12-27,2016.0,12.0,27.0,-0.770370,2.433333,-6.566667,24.922963,0.444444
31582,충북,2016-12-28,2016.0,12.0,28.0,-4.229630,2.722222,-9.688889,2.130370,0.000000
31583,충북,2016-12-29,2016.0,12.0,29.0,-3.951852,-0.070370,-7.859259,2.960370,0.003704
31584,충북,2016-12-30,2016.0,12.0,30.0,-4.485185,2.714815,-11.018519,5.176667,0.000000
